In [6]:
import pandas as pd
import numpy as np
from csv import DictReader as dr


def sigmoid(x):
    return (1/(1+np.exp(-x)))

def computeError(predicted, actual):
    Error = np.mean(predicted != actual)
    return Error * 100

def find_y(Y):
    m=len(Y)
    for i in range(m):
       # print(Y[i])
        if Y[i]>=0.5:
            Y[i]=1
        else:
            Y[i]=0
    return Y

def denormalise_price(price):
    global mean
    global stddev
    #print(price)
    ret = price * stddev[2] + mean[2]
    #print()
    #print(ret)
    return ret




def add_feature(X):
    m=X.shape[0]
    one = np.ones((len(X),6))
    for i in range(m):
        x1_2=X[i][0]**2
        x2_2=X[i][1]**2
        x1_x2=X[i][0]*X[i][1]
        x1_2_x2=(X[i][0]**2)*X[i][1]
        x1_x2_2=(X[i][1]**2)*X[i][0]
        x1_2_x2_2=(X[i][0]**2)*(X[i][1]**2)
        one[i][0]=x1_2
        one[i][1]=x2_2
        one[i][2]=x1_x2
        one[i][3]=x1_2_x2
        one[i][4]=x1_x2_2
        one[i][5]=x1_2_x2_2
    X = np.concatenate((X,one),axis=1)
    return X








def batch_gradient(X_train,Y_train):
    interations=1000
    alpha = 0.01        
    W0 = 0
    W1 = 0
    sz = len(X_train)
            
    costs = []
    W = [[1],[1],[1],[1],[1],[1],[1],[1],[1]]
    #print(X_train.shape)
    for epoc in range(interations):
                
        s=sigmoid(X_train.dot(W))
       # print(s.shape,Y_train.shape)
        d=(s-Y_train)
       # print(d.shape)
        #print(d.shape)
        diff=X_train.T.dot(d)*(alpha/sz)
       # print(diff.shape)
        W = W - diff
        new_Cost = np.sum((sigmoid(X_train.dot(W))-Y_train)**2)/(2*sz)
        costs.append(new_Cost)
    return (costs,W)

def stochastic_gradient(X_train,Y_train):
    interations=1000
    alpha = 0.01        
    W0 = 0
    W1 = 0
    sz = len(X_train)
            
    costs = []
    W = [[1],[1],[1],[1],[1],[1],[1],[1],[1]]
    for epoc in range(interations):  
        cost=0.0
        for j in range(sz):
            rand_int=np.random.randint(0,sz)
            X_i=X_train[rand_int,:].reshape(1,X_train.shape[1])
            Y_i=Y_train[rand_int]
            pred=sigmoid(np.dot(X_i,W))
            W=W-(1/sz)*alpha*(np.dot(np.transpose(X_i),(pred-Y_i)))
        new_Cost = np.sum((sigmoid(X_train.dot(W))-Y_train)**2)/(2*sz)
        costs.append(new_Cost)
    return (costs,W)

def mini_gradient(X_train,Y_train):
    interations=1000
    alpha = 0.01        
    W0 = 0
    W1 = 0
    sz = len(X_train)
            
    costs = []
    W = [[1],[1],[1],[1],[1],[1],[1],[1],[1]]
    #print(X_train.shape)
    for epoc in range(interations):  
        cost=0.0
        num_batch=int(sz/10)
        for i in range(1):
            X_i=X_train[i:i+10]
            Y_i=Y_train[i:i+10]
            pred=sigmoid(np.dot(X_i,W))
            W=W-(1/sz)*alpha*(np.dot(np.transpose(X_i),(pred-Y_i)))
        new_Cost = np.sum((sigmoid(X_train.dot(W))-Y_train)**2)/(2*sz)
        costs.append(new_Cost)
    return (costs,W)

data=pd.read_csv('data.csv')


Y=data.iloc[:,-1:]
X=data.iloc[:,:-1]
Y = np.array(Y)
X = np.array(X)

X=add_feature(X)
one = np.ones((len(X),1))
X = np.concatenate((one,X),axis=1)
totalsize = len(X)
trainsize = int(totalsize*0.7)
Y_train = Y[:trainsize]
Y_test = Y[trainsize:]
X_train = X[:trainsize]
X_test = X[trainsize:]

#print(X_test)
costs,W=batch_gradient(X_train,Y_train)
print("Without Feature Scaling")
print("Batch_Gradient\nParameters:\n",W)
print()


Y_pred=sigmoid(np.dot(X_test,W))
#print(X_test.shape,W.shape)
Y_pred=find_y(Y_pred)
er=computeError(Y_pred,Y_test)
print("Error is :",er,"%\nAccuracy is :",100-er,"%")
costs,W=stochastic_gradient(X_train,Y_train)
print("\nStochastic_Gradient\nParameters:\n",W)


Y_pred=sigmoid(np.dot(X_test,W))
#print(X_test.shape,W.shape)
Y_pred=find_y(Y_pred)
er=computeError(Y_pred,Y_test)
print("Error:",er,"%\nAccuracy:",100-er,"%")
costs,W=mini_gradient(X_train,Y_train)
print("\nMini_Gradient\nParameters:\n",W)


Y_pred=sigmoid(np.dot(X_test,W))
#print(X_test.shape,W.shape)
Y_pred=find_y(Y_pred)
er=computeError(Y_pred,Y_test)
print("Error:",er,"%\nAccuracy:",100-er,"%")

mean = data.mean()
stddev = data.std()
data = (data - data.mean())/data.std()
X=data.iloc[:,:-1]
Y=data.iloc[:,-1:]
X = np.array(X)
Y = np.array(Y)
X=add_feature(X)
one = np.ones((len(X),1))
X = np.concatenate((one,X),axis=1)
totalsize = len(X)
trainsize = int(totalsize*0.7)
X_train = X[:trainsize]
X_test = X[trainsize:]
Y_train = Y[:trainsize]
Y_test = Y[trainsize:]
costs,W=batch_gradient(X_train,Y_train)
print("\nWith Feature Scaling")
print("Batch_Gradient | Parameters  :\n",W)
print()
Y_pred=sigmoid(np.dot(X_test,W))
Y_pred=denormalise_price(Y_pred)


Y_pred=find_y(Y_pred)
#print(Y_pred,Y_test)
er=computeError(Y_pred,denormalise_price(Y_test))
print("Error:",er,"%\nAccuracy:",100-er,"%")
costs,W=stochastic_gradient(X_train,Y_train)
print("\nStochastic_Gradient | Parameters:\n",W)
print()
Y_pred=sigmoid(np.dot(X_test,W))
Y_pred=denormalise_price(Y_pred)
Y_pred=find_y(Y_pred)


#print(Y_pred,Y_test)
er=computeError(Y_pred,denormalise_price(Y_test))
print("Error:",er,"%\nAccuracy:",100-er,"%")
costs,W=stochastic_gradient(X_train,Y_train)
print("\nMini_Gradient | Parameters:\n",W)
print()
Y_pred=sigmoid(np.dot(X_test,W))
Y_pred=denormalise_price(Y_pred)
Y_pred=find_y(Y_pred)


#print(Y_pred,Y_test)
er=computeError(Y_pred,denormalise_price(Y_test))
print("Error:",er,"%\nAccuracy:",100-er,"%")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys


Without Feature Scaling
Batch_Gradient
Parameters:
 [[-1.95628571e+00]
 [-1.28832420e+02]
 [-1.45825527e+02]
 [-6.02256550e+03]
 [-7.64363689e+03]
 [-5.30775940e+03]
 [-1.67378999e+05]
 [-1.98134476e+05]
 [ 1.47766082e+05]]

Error is : 20.0 %
Accuracy is : 80.0 %

Stochastic_Gradient
Parameters:
 [[-5.11285714e-01]
 [-6.65139531e+01]
 [-7.20980015e+01]
 [-3.19358288e+03]
 [-3.62176912e+03]
 [-2.69702041e+03]
 [-8.90296129e+04]
 [-9.44769371e+04]
 [ 2.44519560e+03]]
Error: 40.0 %
Accuracy: 60.0 %

Mini_Gradient
Parameters:
 [[ 6.00571429e-01]
 [-9.92446933e+00]
 [-2.31487329e+01]
 [-1.38520980e+02]
 [-1.49591552e+03]
 [-6.31191344e+02]
 [-5.48552708e+03]
 [-3.57749046e+04]
 [ 9.32878535e+03]]
Error: 20.0 %
Accuracy: 80.0 %

With Feature Scaling
Batch_Gradient | Parameters  :
 [[-2.92132139]
 [ 6.30532193]
 [ 4.78147166]
 [-5.37483094]
 [-3.55299645]
 [ 0.63168983]
 [ 3.06224537]
 [ 3.58763317]
 [-6.26313748]]

Error: 20.0 %
Accuracy: 80.0 %

Stochastic_Gradient | Parameters:
 [[-2.96273